### Load preprocessed data

In [1]:
import numpy as np
fh = np.load('../data/dataset.npz')
# We have a bunch of feature columns and last column is the y-target
train_x = fh['train_x'].astype(np.int64)
train_y = fh['train_y']

test_x = fh['test_x'].astype(np.int64)
test_y = fh['test_y']

n_user = int(fh['n_user'])
n_item = int(fh['n_item'])
n_occu = int(fh['n_occu'])
n_rank = int(fh['n_ranks'])

### Define the MF Model

In [2]:
import torch
from torch import nn
import torch.nn.functional as F

def l2_regularize(array):
    loss = torch.sum(array ** 2.0)
    return loss


def total_variation(array):
    return torch.sum(torch.abs(array[:, :-1] - array[:, 1:]))


class MF(nn.Module):
    itr = 0
    
    def __init__(self, n_user, n_item, n_occu, n_rank, k=18, kt=2, c_temp=1.0,
                 c_vector=1.0, c_bias=1.0, c_ut=1.0, writer=None):
        super(MF, self).__init__()
        self.writer = writer
        self.k = k
        self.n_user = n_user
        self.n_item = n_item
        self.c_bias = c_bias
        self.c_vector = c_vector
        self.c_temp = c_temp
        self.c_ut = c_ut
        self.user = nn.Embedding(n_user, k)
        self.item = nn.Embedding(n_item, k)
        self.bias_user = nn.Embedding(n_user, 1)
        self.bias_item = nn.Embedding(n_item, 1)
        self.bias = nn.Parameter(torch.ones(1))
        self.occu = nn.Embedding(n_occu, k)
        self.temp = nn.Embedding(n_rank, k)
        
        # **NEW: temporal vectors
        # user_temp = m_u
        self.user_temp = nn.Embedding(n_user, kt)
        # temp = n_t
        self.temp = nn.Embedding(n_rank, kt)

    
    def __call__(self, train_x):
        user_id = train_x[:, 0]
        item_id = train_x[:, 1]
        occu_id = train_x[:, 3]
        vector_user = self.user(user_id)
        vector_item = self.item(item_id)
        vector_occu = self.occu(occu_id)
        bias_user = self.bias_user(user_id).squeeze()
        bias_item = self.bias_item(item_id).squeeze()
        biases = (self.bias + bias_user + bias_item)
        ui_interaction = torch.sum(vector_user * vector_item, dim=1)
        oi_interaction = torch.sum(vector_user * vector_occu, dim=1)
        
        # **NEW: user-time interaction
        rank = train_x[:, 2]
        vector_user_temp = self.user_temp(user_id)
        vector_temp = self.temp(rank)
        ut_interaction = torch.sum(vector_user_temp * vector_temp, dim=1)
        
        prediction = ui_interaction + oi_interaction + ut_interaction + biases
        return prediction
    
    def loss(self, prediction, target):
        loss_mse = F.mse_loss(prediction.squeeze(), target.squeeze())
        prior_bias_user =  l2_regularize(self.bias_user.weight) * self.c_bias
        prior_bias_item = l2_regularize(self.bias_item.weight) * self.c_bias
        prior_user =  l2_regularize(self.user.weight) * self.c_vector
        prior_item = l2_regularize(self.item.weight) * self.c_vector
        prior_occu = l2_regularize(self.occu.weight) * self.c_vector
        
        # New: total variation regularization
        prior_ut = l2_regularize(self.user_temp.weight) * self.c_ut
        prior_tv = total_variation(self.temp.weight) * self.c_temp
        
        total = (loss_mse + prior_user + prior_item + prior_ut +
                 prior_bias_item + prior_bias_user +  prior_occu + prior_tv)
        for name, var in locals().items():
            if type(var) is torch.Tensor and var.nelement() == 1 and self.writer is not None:
                self.writer.add_scalar(name, var, self.itr)
        return total

### Train model

In [3]:
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Loss
from tensorboardX import SummaryWriter
from ignite.metrics import MeanSquaredError

from loader import Loader
from datetime import datetime

#### Hyperparameters

In [4]:
lr = 1e-2
k = 10
kt = 2
c_bias = 1e-6
c_vector = 1e-6
c_temp = 1e-6
c_ut = 1e-6

log_dir = 'runs/simple_mf_05_temporal_' + str(datetime.now()).replace(' ', '_')
print(log_dir)

runs/simple_mf_05_temporal_2018-08-19_12:48:09.957336


In [5]:
writer = SummaryWriter(log_dir=log_dir)
model = MF(n_user, n_item, n_occu, n_rank, k=k, kt=kt, c_bias=c_bias, 
           c_vector=c_vector, c_ut=c_ut, c_temp=c_temp, writer=writer)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
trainer = create_supervised_trainer(model, optimizer, model.loss)
metrics = {'accuracy': MeanSquaredError()}
evaluat = create_supervised_evaluator(model, metrics=metrics)
train_loader = Loader(train_x, train_y, batchsize=1024)
test_loader = Loader(test_x, test_y, batchsize=1024)


def log_training_loss(engine, log_interval=400):
    epoch = engine.state.epoch
    itr = engine.state.iteration
    fmt = "Epoch[{}] Iteration[{}/{}] Loss: {:.2f}"
    msg = fmt.format(epoch, itr, len(train_loader), engine.state.output)
    model.itr = itr
    if itr % log_interval == 0:
        print(msg)

trainer.add_event_handler(event_name=Events.ITERATION_COMPLETED, handler=log_training_loss)

def log_validation_results(engine):
    evaluat.run(test_loader)
    metrics = evaluat.state.metrics
    avg_accuracy = metrics['accuracy']
    print("Epoch[{}] Validation MSE: {:.2f} "
          .format(engine.state.epoch, avg_accuracy))
    writer.add_scalar("validation/avg_accuracy", avg_accuracy, engine.state.epoch)

trainer.add_event_handler(event_name=Events.EPOCH_COMPLETED, handler=log_validation_results)


model

MF(
  (user): Embedding(6041, 10)
  (item): Embedding(3953, 10)
  (bias_user): Embedding(6041, 1)
  (bias_item): Embedding(3953, 1)
  (occu): Embedding(21, 10)
  (temp): Embedding(2315, 10)
  (user_temp): Embedding(6041, 10)
)

#### Run model

In [6]:
trainer.run(train_loader, max_epochs=250)

Epoch[1] Iteration[400/879] Loss: 4.56
Epoch[1] Iteration[800/879] Loss: 1.85
Epoch[1] Validation MSE: 1.71 
Epoch[2] Iteration[1200/879] Loss: 1.21
Epoch[2] Iteration[1600/879] Loss: 1.03
Epoch[2] Validation MSE: 1.03 
Epoch[3] Iteration[2000/879] Loss: 0.88
Epoch[3] Iteration[2400/879] Loss: 0.92
Epoch[3] Validation MSE: 0.95 
Epoch[4] Iteration[2800/879] Loss: 0.92
Epoch[4] Iteration[3200/879] Loss: 0.87
Epoch[4] Validation MSE: 0.92 
Epoch[5] Iteration[3600/879] Loss: 0.78
Epoch[5] Iteration[4000/879] Loss: 0.79
Epoch[5] Validation MSE: 0.91 
Epoch[6] Iteration[4400/879] Loss: 0.73
Epoch[6] Iteration[4800/879] Loss: 0.81
Epoch[6] Iteration[5200/879] Loss: 0.79
Epoch[6] Validation MSE: 0.88 
Epoch[7] Iteration[5600/879] Loss: 0.78
Epoch[7] Iteration[6000/879] Loss: 0.79
Epoch[7] Validation MSE: 0.87 
Epoch[8] Iteration[6400/879] Loss: 0.70
Epoch[8] Iteration[6800/879] Loss: 0.75
Epoch[8] Validation MSE: 0.86 
Epoch[9] Iteration[7200/879] Loss: 0.67
Epoch[9] Iteration[7600/879] Loss:

Epoch[67] Validation MSE: 0.86 
Epoch[68] Iteration[59200/879] Loss: 0.62
Epoch[68] Iteration[59600/879] Loss: 0.68
Epoch[68] Validation MSE: 0.86 
Epoch[69] Iteration[60000/879] Loss: 0.59
Epoch[69] Iteration[60400/879] Loss: 0.61
Epoch[69] Validation MSE: 0.87 
Epoch[70] Iteration[60800/879] Loss: 0.59
Epoch[70] Iteration[61200/879] Loss: 0.64
Epoch[70] Validation MSE: 0.87 
Epoch[71] Iteration[61600/879] Loss: 0.56
Epoch[71] Iteration[62000/879] Loss: 0.61
Epoch[71] Iteration[62400/879] Loss: 0.61
Epoch[71] Validation MSE: 0.87 
Epoch[72] Iteration[62800/879] Loss: 0.60
Epoch[72] Iteration[63200/879] Loss: 0.63
Epoch[72] Validation MSE: 0.87 
Epoch[73] Iteration[63600/879] Loss: 0.66
Epoch[73] Iteration[64000/879] Loss: 0.68
Epoch[73] Validation MSE: 0.86 
Epoch[74] Iteration[64400/879] Loss: 0.53
Epoch[74] Iteration[64800/879] Loss: 0.68
Epoch[74] Validation MSE: 0.86 
Epoch[75] Iteration[65200/879] Loss: 0.58
Epoch[75] Iteration[65600/879] Loss: 0.65
Epoch[75] Validation MSE: 0.87

Epoch[132] Validation MSE: 0.86 
Epoch[133] Iteration[116400/879] Loss: 0.63
Epoch[133] Iteration[116800/879] Loss: 0.67
Epoch[133] Validation MSE: 0.87 
Epoch[134] Iteration[117200/879] Loss: 0.60
Epoch[134] Iteration[117600/879] Loss: 0.63
Epoch[134] Validation MSE: 0.87 
Epoch[135] Iteration[118000/879] Loss: 0.55
Epoch[135] Iteration[118400/879] Loss: 0.62
Epoch[135] Validation MSE: 0.87 
Epoch[136] Iteration[118800/879] Loss: 0.57
Epoch[136] Iteration[119200/879] Loss: 0.64
Epoch[136] Validation MSE: 0.87 
Epoch[137] Iteration[119600/879] Loss: 0.57
Epoch[137] Iteration[120000/879] Loss: 0.54
Epoch[137] Iteration[120400/879] Loss: 0.64
Epoch[137] Validation MSE: 0.87 
Epoch[138] Iteration[120800/879] Loss: 0.57
Epoch[138] Iteration[121200/879] Loss: 0.66
Epoch[138] Validation MSE: 0.87 
Epoch[139] Iteration[121600/879] Loss: 0.61
Epoch[139] Iteration[122000/879] Loss: 0.64
Epoch[139] Validation MSE: 0.87 
Epoch[140] Iteration[122400/879] Loss: 0.58
Epoch[140] Iteration[122800/879]

KeyboardInterrupt: 